In [1]:
import sys
import requests
import os
from pathlib import Path
from random import randint, random
from database import Persistence, CursorFromConnectionFromPool
from datetime import datetime
import pandas as pd
from psycopg2 import sql
import json
import utils
from bs4 import BeautifulSoup
from psycopg2.extras import RealDictCursor
import psycopg2


base_link = 'https://www.ourcommons.ca/members/en/'

# with CursorFromConnectionFromPool() as cur:
#     try:
#         legislator_data = pd.read_sql("SELECT * FROM ca_fed_legislators WHERE party='Liberal'", cur.connection)
#         # legislation_data = pd.read_sql("SELECT * FROM ca_fed_legislation WHERE pm_party='Liberal'", cur.connection)
#         cur.connection.commit()
#     except Exception as e:
#         print(f'An exception occured executing a query: \n{e}')

# params = list(legislator_data['source_id'])

db_host = 'openparl.cia2zobysfwo.us-west-2.rds.amazonaws.com'
db_port = '5432'
db_name = 'videoanalysis'
db_pass = 'Dedenne#3'
db_user = 's_leung'

with psycopg2.connect(host=db_host, password=db_pass, user=db_user, database=db_name, port='5432') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        try:
            completed_dataset = pd.read_sql("SELECT * FROM completed_dataset", cur.connection)
            cur.connection.commit()
        except Exception as e:
            print(f'GG NO RE: {e}')

In [2]:
# commons_link = 'https://www.ourcommons.ca/publicationsearch/en/?per=88867&pubType=37&View=D'

# # for param in params:
# #     print(requests.get(f'https://www.ourcommons.ca/publicationsearch/en/?per={param}&pubType=37&View=D'))

# response = requests.get(commons_link)
# soup = BeautifulSoup(response.content, 'lxml')
# div_lst = soup.find('div', {'id':'Publications'}).find_all('div', {'class':'Publication PubType37'})

# counter = {}

# for item in div_lst:
#     topics = [x.text for x in item.find('div', {'class':'pi-indexes'}).find_all('a')]
#     for topic in topics:
#         if topic in counter:
#             counter[topic] += 1
#         else:
#             counter[topic] = 1

In [3]:
completed_dataset_dict = completed_dataset.to_dict('records')
len(completed_dataset_dict)

380804

In [4]:
# persons = list(pd.DataFrame(completed_dataset_dict)['who_en'])
# counter = {}
# for person in persons:
#     if type(person)==str:
#         if '(' in person:
#             try:
#                 party = person.split('(')[0]
#                 if party in counter:
#                     counter[party]+=1
#                 else:
#                     counter[party]=1
#             except IndexError:
#                 pass 

# names = [key.split('.')[1].strip() for key, value in counter.items() if '.' in key]

In [5]:
import pickle
with open(r"data_map.pkl", "rb") as input_file:
     data = pickle.load(input_file)

new_map = {}

data_lst = [{key: value} for key, value in data.items()]

for i in range(len(data_lst)):
    for key, value in data_lst[i].items():
        id_set = value[1]
        for source_id in id_set:
            new_map[source_id] = {
                'name':value[0],
                'goverlytics_id':key
            }

In [28]:
legis_topics = {}

for item in completed_dataset_dict:
    if item['member_id'] in new_map:
        # print(new_map[item['member_id']])
        # print(item['cap_topic'])
        # print('\n')
        database_dict = new_map[item['member_id']]
        if database_dict['goverlytics_id'] in legis_topics:
            if item['cap_topic'] in legis_topics[database_dict['goverlytics_id']]['topics']:
                legis_topics[database_dict['goverlytics_id']]['topics'][item['cap_topic']] += 1
            else:
                legis_topics[database_dict['goverlytics_id']]['topics'][item['cap_topic']] = 1
        else:
            legis_topics[database_dict['goverlytics_id']] = {
                'name': database_dict['name'],
                'topics': {
                    item['cap_topic']: 1
                }
            }
        

In [30]:
legis_topics_lst = [{'name': value['name'], 'goverlytics_id': key, 'topics': value['topics']} for key, value in legis_topics.items()]


In [42]:
import utils
from datetime import date, datetime

table = 'ca_fed_parliament_topics'
data = legis_topics_lst

def json_serial(obj):
    """
    Serializes objects so they may be placed into JSON format.

    Author: Jay Taylor
    Source: https://stackoverflow.com/questions/11875770/how-to-overcome-datetime-datetime-not-json-serializable
    """
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))


with CursorFromConnectionFromPool() as cur:
    try:
        create_table_query = sql.SQL("""

            CREATE TABLE IF NOT EXISTS {table} (
                name text,
                goverlytics_id bigint UNIQUE,
                topics json
            );

            ALTER TABLE {table} OWNER TO rds_ad;
        """).format(table=sql.Identifier(table))

        cur.execute(create_table_query)
        cur.connection.commit()
    except Exception as e:
        print(f'An exception occured executting a query:\n{e}')
        cur.connection.rollback()

    insert_legislator_query = sql.SQL("""
            INSERT INTO {table}
            VALUES (%s, %s, %s)
            ON CONFLICT (goverlytics_id) DO UPDATE SET
                name = excluded.name,
                goverlytics_id = excluded.goverlytics_id,
                topics = excluded.topics
            """).format(table=sql.Identifier(table))

    # This is used to convert dictionaries to rows. Need to test it out!
    for item in data:
        # if isinstance(item, dict):
        #     item = utils.DotDict(item)
        try:
            tup = (
                item['name'],
                item['goverlytics_id'],
                json.dumps(item['topics'], default=json_serial)
            )

            cur.execute(insert_legislator_query, tup)
        except Exception as e:
            print(f'Exception occured inserting the following data:\n{tup}')
            print(e)
            cur.connection.rollback()

print('done!')

done!


In [44]:
x = 'ak, al, ar, az, co, de, hi, ia, id, il, ks, ky, la, ma, md, mi, nc, nd, ne, nm, nv, oh, ok, pa, ri, sc, sd, tn, ut, va, vt, wi, wv, wy'
len(x.split(','))

34